<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/embeddings/huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Local Embeddings with HuggingFace

LlamaIndex has support for HuggingFace embedding models, including BGE, Instructor, and more.

Furthermore, we provide utilities to create and use ONNX models using the [Optimum library](https://huggingface.co/docs/transformers/serialization#exporting-a-transformers-model-to-onnx-with-optimumonnxruntime) from HuggingFace.

## HuggingFaceEmbedding

The base `HuggingFaceEmbedding` class is a generic wrapper around any HuggingFace model for embeddings. All [embedding models](https://huggingface.co/models?library=sentence-transformers) on Hugging Face should work. You can refer to the [embeddings leaderboard](https://huggingface.co/spaces/mteb/leaderboard) for more recommendations.

This class depends on the sentence-transformers package, which you can install with `pip install sentence-transformers`.

NOTE: if you were previously using a `HuggingFaceEmbeddings` from LangChain, this should give equivalent results.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [6]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.2.1
    Uninstalling sentence-transformers-3.2.1:
      Successfully uninstalled sentence-transformers-3.2.1


In [7]:
!pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.0/189.0 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 24.6 MB/s eta 0:00:00


In [8]:
%cd /content/agentless_s
!mkdir testbed

[Errno 2] No such file or directory: '/content/agentless_s'
/content


In [9]:
!git clone https://github.com/django/django.git testbed/django
!git clone https://github.com/sympy/sympy.git testbed/sympy
!git clone https://github.com/matplotlib/matplotlib.git testbed/matplotlib
!git clone https://github.com/scikit-learn/scikit-learn.git testbed/scikit-learn
!git clone https://github.com/pytest-dev/pytest.git testbed/pytest
!git clone https://github.com/sphinx-doc/sphinx.git testbed/sphinx
!git clone https://github.com/psf/requests.git testbed/requests
!git clone https://github.com/pylint-dev/pylint.git testbed/pylint
!git clone https://github.com/astropy/astropy.git testbed/astropy
!git clone https://github.com/pydata/xarray.git testbed/xarray
!git clone https://github.com/mwaskom/seaborn.git testbed/seaborn
!git clone https://github.com/pallets/flask.git testbed/flask

Cloning into 'testbed/django'...
remote: Enumerating objects: 534173, done.
remote: Counting objects: 100% (474/474), done.
remote: Compressing objects: 100% (283/283), done.
remote: Total 534173 (delta 291), reused 291 (delta 190), pack-reused 533699 (from 1)
Receiving objects: 100% (534173/534173), 258.23 MiB | 31.65 MiB/s, done.
Resolving deltas: 100% (383307/383307), done.
Cloning into 'testbed/sympy'...
remote: Enumerating objects: 410069, done.
remote: Counting objects: 100% (2065/2065), done.
remote: Compressing objects: 100% (1009/1009), done.
remote: Total 410069 (delta 1348), reused 1581 (delta 1050), pack-reused 408004 (from 1)
Receiving objects: 100% (410069/410069), 183.53 MiB | 34.41 MiB/s, done.
Resolving deltas: 100% (326912/326912), done.
Cloning into 'testbed/matplotlib'...
remote: Enumerating objects: 332527, done.
remote: Counting objects: 100% (2009/2009), done.
remote: Compressing objects: 100% (1464/1464), done.
remote: Total 332527 (delta 1122), reused 1205 (del

In [6]:
# Utility Functions
import subprocess
def checkout_commit(repo_path, commit_id):
    """Checkout the specified commit in the given local git repository.
    First discards any untracked changes in the repository.
    :param repo_path: Path to the local git repository
    :param commit_id: Commit ID to checkout
    :return: None
    """
    try:
        # Clean untracked files and directories
        print(f"Cleaning untracked files in repository at {repo_path}...")
        subprocess.run(["git", "-C", repo_path, "clean", "-fd"], check=True)

        # Discard changes in tracked files
        print("Discarding changes in tracked files...")
        subprocess.run(["git", "-C", repo_path, "reset", "--hard"], check=True)

        # Change directory to the provided repository path and checkout the specified commit
        print(f"Checking out commit {commit_id}...")
        subprocess.run(["git", "-C", repo_path, "checkout", commit_id], check=True)
        print("Commit checked out successfully.")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while running git command: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


def get_all_file_paths(folder_path, base_dir):
    """
    Returns a list of absolute file paths for all files in the given folder and its subfolders.

    Args:
        folder_path (str): The path to the folder to search

    Returns:
        list: A list of absolute file paths
    """
    file_paths = []

    # Convert to absolute path if relative path is given
    folder_path = os.path.abspath(folder_path)

    # Check if the folder exists
    if not os.path.exists(folder_path):
        raise ValueError(f"The folder '{folder_path}' does not exist")

    # Walk through the directory tree
    for root, dirs, files in os.walk(folder_path):
        # Add the absolute path of each file to the list
        for file in files:
            file_path = os.path.join(root, file)
            repo_file_path = os.path.relpath(file_path, folder_path)
            file_paths.append(repo_file_path)

    return file_paths

In [12]:
# Main retrieve function
import os
from llama_index.core import (
    Document,
    StorageContext,
    VectorStoreIndex,
    load_index_from_storage,
)
from llama_index.core.retrievers import VectorIndexRetriever

def retrieve(
    files,
    prompt,
    embedding_model,
    save_dir,
    top_k,
    file_to_contents,
):
    # Check if index files exist in save_dir
    if os.path.exists(save_dir) and os.path.isdir(save_dir) and os.listdir(save_dir):
        storage_context = StorageContext.from_defaults(persist_dir=save_dir)
        index = load_index_from_storage(storage_context)
        print("Loading existing index from storage")
    else:
        print("Creating new index")
        documents = []
        for file in files:
            file_content = file_to_contents[file]
            meta_data = {
                "File Name": file,
            }
            doc = Document(
                text=file_content,
                metadata=meta_data,
                metadata_template="{key}: {value}",
                text_template="{metadata_str}\n-----\nCode:\n{content}",
            )
            documents.append(doc)

        index = VectorStoreIndex.from_documents(documents, embed_model=embedding_model) #embedding_model)
        index.storage_context.persist(persist_dir=save_dir)

    retriever = VectorIndexRetriever(index=index, similarity_top_k=top_k)
    retrieved_documents = retriever.retrieve(prompt)

    file_names = []
    file_contents = []
    for node in retrieved_documents:
        file = node.metadata["File Name"]
        if file not in file_names:
            file_names.append(file)
            file_contents.append(file_to_contents[file])

    return file_names, file_contents

In [13]:
!pip install xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import os
HF_TOKEN = os.getenv("HUGGING_FACE_TOKEN")
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
RETRIEVAL_MODEL = HuggingFaceEmbedding(model_name="dunzhang/stella_en_400M_v5", trust_remote_code=True)

NameError: name 'os' is not defined

In [13]:
import concurrent
import json
import os
import threading
from concurrent.futures import ThreadPoolExecutor
from random import shuffle
import sys
import pandas as pd
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

repo_to_top_folder = {
    "django/django": "django",
    "sphinx-doc/sphinx": "sphinx",
    "scikit-learn/scikit-learn": "scikit-learn",
    "sympy/sympy": "sympy",
    "pytest-dev/pytest": "pytest",
    "matplotlib/matplotlib": "matplotlib",
    "astropy/astropy": "astropy",
    "pydata/xarray": "xarray",
    "mwaskom/seaborn": "seaborn",
    "psf/requests": "requests",
    "pylint-dev/pylint": "pylint",
    "pallets/flask": "flask",
}

# Create locks for each repo to prevent concurrent access
repo_locks = {repo: threading.Lock() for repo in repo_to_top_folder.keys()}
output_lock = threading.Lock()

splits = {
    "dev": "data/dev-00000-of-00001.parquet",
    "test": "data/test-00000-of-00001.parquet",
}
swe_df = pd.read_parquet("hf://datasets/princeton-nlp/SWE-bench_Lite/" + splits["test"])

RETRIEVAL_INSTRUCTION = (
    "\n\nFind code the code which need to be edited to solve the above issue"
)

HF_TOKEN = os.getenv("HUGGING_FACE_TOKEN")
RETRIEVAL_MODEL =HuggingFaceEmbedding(model_name="dunzhang/stella_en_400M_v5", trust_remote_code=True)

def retrieve_instance(args, row, output_path, existing_ids):
    """Process a single row from the DataFrame"""
    # Skip if instance_id already exists
    if row["instance_id"] in existing_ids:
        return None
    print(row["instance_id"])
    original_prompt = row["problem_statement"]
    persist_dir = os.path.join(
        args.base_path, "fl", "embeddings_cornstack", row["instance_id"]
    )

    file_dir = os.path.join(args.testbed_dir, repo_to_top_folder[row["repo"]])
    full_prompt = original_prompt + RETRIEVAL_INSTRUCTION

    with repo_locks[row["repo"]]:
        checkout_commit(file_dir, row["base_commit"])
        files = get_all_file_paths(file_dir, args.testbed_dir)

        files = [
            file_path
            for file_path in files
            if file_path.endswith(".py") and (not "/test" in file_path)
        ]
        file_to_contents = {}
        for file in files:
            with open(os.path.join(file_dir, file), "r") as f:
                try:
                    file_to_contents[file] = f.read()
                except:
                    print("Error in reading file")

    file_names, file_contents = retrieve(
        list(file_to_contents.keys()),
        full_prompt,
        RETRIEVAL_MODEL,
        persist_dir,
        args.retrieve_num,
        file_to_contents
    )

    result = {
        "instance_id": row["instance_id"],
        "problem_description": original_prompt,
        "found_files": file_names,
        "file_contents": file_contents,
    }

    # Write the result to the output file with a lock to prevent concurrent writing
    with output_lock:
        with open(output_path, "a") as f:
            json_line = json.dumps(result)
            f.write(json_line + "\n")
            f.flush()

    return result

class Args:
    def __init__(self):
        self.base_path = "/content/"
        self.testbed_dir = "/content/testbed"
        self.retrieve_num = 50
        self.output_file = "retrieved_mbet.jsonl"
        self.max_workers = 1
        self.embedding_dir = "mbet_embeddings"

args = Args()

output_path = os.path.join(args.base_path, args.output_file)
# Get existing instance IDs
existing_ids = get_existing_instance_ids(output_path)
print(existing_ids)

# Create file if it doesn't exist
if not os.path.exists(output_path):
    with open(output_path, "w") as f:
        pass

if args.max_workers == 1:
    for _, row in swe_df.iterrows():
        retrieve_instance(args, row, output_path, existing_ids)
else:
    with ThreadPoolExecutor(max_workers=args.max_workers) as executor:
        rows_list = list(swe_df.iterrows())
        shuffle(rows_list)

        future_to_row = {
            executor.submit(retrieve_instance, args, row, output_path, existing_ids): row
            for _, row in rows_list
        }

        for future in concurrent.futures.as_completed(future_to_row):
            row = future_to_row[future]
            try:
                result = future.result()
            except Exception as e:
                print(f"Error processing row: {e}")




Some weights of the model checkpoint at dunzhang/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/sentence_transformers/models/Dense.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECU

path exists






































































































































































































































































































{'django__django-15814', 'django__django-16046', 'django__django-14608', 'pydata__xarray-4248', 'pytest-dev__pytest-5221', 'django__django-12113', 'scikit-learn__scikit-learn-10297', 'scikit-learn__scikit-learn-13584', 'sympy__sympy-13647', 'sympy__sympy-12171', 'django__django-13658', 'django__django-14787', 'django__django-16255', 'astropy__astropy-12907', 'pytest-dev__pytest-7168', 'matplotlib__matplotlib-22835', 'sympy__sympy-15308', 'sympy__sympy-13895', 'sympy__sympy-17630', 'matplotlib__matplotlib-25332', 'sympy__sympy-21055', 'scikit-learn__scikit-learn-14092', 'sympy__sympy-18189', 'pytest-dev__pytest-8365', 'django__django-14730', 'django__django-13321', 'sphinx-doc__sphinx-8

In [3]:
import os
import json
output_path = "/content/retrieved_mbet.jsonl"
def get_existing_instance_ids(output_path):
    """Read existing instance IDs from the output file"""
    existing_ids = set()
    if os.path.exists(output_path):
        print("path exists")
        with open(output_path, 'r') as f:
            for line in f:
                data = json.loads(line.strip())
                print()
                existing_ids.add(data['instance_id'])
    return existing_ids
existing_ids = get_existing_instance_ids(output_path)
print(existing_ids)

path exists






































































































































































































































































































{'django__django-15814', 'django__django-16046', 'django__django-14608', 'pydata__xarray-4248', 'pytest-dev__pytest-5221', 'django__django-12113', 'scikit-learn__scikit-learn-10297', 'scikit-learn__scikit-learn-13584', 'sympy__sympy-13647', 'sympy__sympy-12171', 'django__django-13658', 'django__django-14787', 'django__django-16255', 'astropy__astropy-12907', 'pytest-dev__pytest-7168', 'matplotlib__matplotlib-22835', 'sympy__sympy-15308', 'sympy__sympy-13895', 'sympy__sympy-17630', 'matplotlib__matplotlib-25332', 'sympy__sympy-21055', 'scikit-learn__scikit-learn-14092', 'sympy__sympy-18189', 'pytest-dev__pytest-8365', 'django__django-14730', 'django__django-13321', 'sphinx-doc__sphinx-8

In [9]:
print(output_path)

/content/retrieved_mbet.jsonl


In [ ]:
embeddings = embed_model.get_text_embedding("Hello World!")
print(len(embeddings))
print(embeddings[:5])

384
[-0.003275713650509715, -0.011690733022987843, 0.041559211909770966, -0.038148071616888046, 0.024183085188269615]


## InstructorEmbedding

Instructor Embeddings are a class of embeddings specifically trained to augment their embeddings according to an instruction. By default, queries are given `query_instruction="Represent the question for retrieving supporting documents: "` and text is given `text_instruction="Represent the document for retrieval: "`.

They rely on the `Instructor` and `SentenceTransformers` (version 2.2.2) pip package, which you can install with `pip install InstructorEmbedding` and `pip install -U sentence-transformers==2.2.2`.

In [ ]:
from llama_index.embeddings.instructor import InstructorEmbedding

embed_model = InstructorEmbedding(model_name="hkunlp/instructor-base")

TypeError: INSTRUCTOR._load_sbert_model() got an unexpected keyword argument 'token'

In [ ]:
embeddings = embed_model.get_text_embedding("Hello World!")
print(len(embeddings))
print(embeddings[:5])

768
[ 0.02155361 -0.06098218  0.01796207  0.05490903  0.01526906]


## OptimumEmbedding

Optimum in a HuggingFace library for exporting and running HuggingFace models in the ONNX format.

You can install the dependencies with `pip install transformers optimum[exporters]`.

First, we need to create the ONNX model. ONNX models provide improved inference speeds, and can be used across platforms (i.e. in TransformersJS)

In [ ]:
from llama_index.embeddings.huggingface_optimum import OptimumEmbedding

OptimumEmbedding.create_and_save_optimum_model(
    "BAAI/bge-small-en-v1.5", "./bge_onnx"
)

/home/loganm/miniconda3/envs/llama-index/lib/python3.11/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Framework not specified. Using pt to export to ONNX.
Using the export variant default. Available variants are:
	- default: The default ONNX variant.
Using framework PyTorch: 2.0.1+cu117
Overriding 1 configuration item(s)
	- use_cache -> False


============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

Saved optimum model to ./bge_onnx. Use it with `embed_model = OptimumEmbedding(folder_name='./bge_onnx')`.


In [ ]:
embed_model = OptimumEmbedding(folder_name="./bge_onnx")

In [ ]:
embeddings = embed_model.get_text_embedding("Hello World!")
print(len(embeddings))
print(embeddings[:5])

384
[-0.10364960134029388, -0.20998482406139374, -0.01883639395236969, -0.5241696834564209, 0.0335749015212059]


## Benchmarking

Let's try comparing using a classic large document -- the IPCC climate report, chapter 3.

In [ ]:
!curl https://www.ipcc.ch/report/ar6/wg2/downloads/report/IPCC_AR6_WGII_Chapter03.pdf --output IPCC_AR6_WGII_Chapter03.pdf

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20.7M  100 20.7M    0     0  16.5M      0  0:00:01  0:00:01 --:--:-- 16.5M


In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings

documents = SimpleDirectoryReader(
    input_files=["IPCC_AR6_WGII_Chapter03.pdf"]
).load_data()

### Base HuggingFace Embeddings

In [ ]:
import os
import openai

# needed to synthesize responses later
os.environ["OPENAI_API_KEY"] = "sk-..."
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# loads BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
test_emeds = embed_model.get_text_embedding("Hello World!")

Settings.embed_model = embed_model

In [ ]:
%%timeit -r 1 -n 1
index = VectorStoreIndex.from_documents(documents, show_progress=True)

Parsing documents into nodes:   0%|          | 0/172 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/428 [00:00<?, ?it/s]

1min 27s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Optimum Embeddings

We can use the onnx embeddings we created earlier

In [ ]:
from llama_index.embeddings.huggingface_optimum import OptimumEmbedding

embed_model = OptimumEmbedding(folder_name="./bge_onnx")
test_emeds = embed_model.get_text_embedding("Hello World!")

Settings.embed_model = embed_model

In [ ]:
%%timeit -r 1 -n 1
index = VectorStoreIndex.from_documents(documents, show_progress=True)

Parsing documents into nodes:   0%|          | 0/172 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/428 [00:00<?, ?it/s]

1min 9s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
